In [1]:
# import stuff
%load_ext autoreload
%autoreload 2
%matplotlib inline

from random import randint

import numpy as np
import torch

In [3]:
# Load model
from models import InferSent
model_version = 2
MODEL_PATH = "encoder/infersent%s.pkl" % model_version
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0, 'version': model_version}
model = InferSent(params_model)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [4]:
# Keep it on CPU or put it on GPU
use_cuda = False
model = model.cuda() if use_cuda else model

In [5]:
# If infersent1 -> use GloVe embeddings. If infersent2 -> use InferSent embeddings.
W2V_PATH = 'GloVe/glove.840B.300d.txt' if model_version == 1 else 'fastText/crawl-300d-2M.vec'
model.set_w2v_path(W2V_PATH)

In [6]:
# Load embeddings of K most frequent words
model.build_vocab_k_words(K=100000)

Vocab size : 100000


In [19]:
def similarity_score(question1, question2, embeddings_model, similarity_model):
    
    embeddings_1 = embeddings_model.encode([question1], bsize=128, tokenize=False, verbose=True)
    embeddings_2 = embeddings_model.encode([question2], bsize=128, tokenize=False, verbose=True)
    
    score = similarity_model.predict([embeddings_1, embeddings_2])
    
    return score[0][0]

In [23]:
from keras.models import load_model
similarity_model = load_model('trained_model.h5')
q1 = 'Which fish would survive in salt water?'
q2 = 'How was your first day at IIT?'
q3 = 'How was the first day at IIT college?'
score = similarity_score(q3, q2, model, similarity_model)
print(score)

Nb words kept : 9/10 (90.0%)
Speed : 4.1 sentences/s (cpu mode, bsize=128)
Nb words kept : 8/9 (88.9%)
Speed : 11.1 sentences/s (cpu mode, bsize=128)
0.0280019
